In [ ]:
import numpy as np
import ast
from keras.models import Sequential
from keras.optimizers import Adam
import pandas as pd
from keras.layers import Dense, Dropout, BatchNormalization
from google.colab import drive
from keras.models import Model
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier


drive.mount('/content/drive')

In [ ]:
#ENTER PATH TO THE DATA
data = pd.read_excel("/content/drive/MyDrive/AI-Project-3/model1_data_final.xlsx")

ship = data['ship']
alien_prob = data['beliefNetworkAlien']
crew_prob = data['beliefNetworkCrew']
bot = data['bot_cell']
alien_beep = data["isBeepAlien"]
crew_beep = data["isBeepCrew"]

move = data["move"].tolist()

In [ ]:
print("Data size: ", len(data))

X_train = []
y_train = []
X_test = []
y_test = []


for i in range(len(data)-1):
  a = ast.literal_eval(ship[i])
  b = list(ast.literal_eval(alien_prob[i][1:-1]))
  c = list(ast.literal_eval(crew_prob[i][1:-1]))
  d = ast.literal_eval(bot[i])
  e = [float(alien_beep[i])]
  f = [float(crew_beep[i])]

  X_train.append(a+b+c+d+e+f)
  y_train.append(float(move[i]))

X_train = np.array(X_train)
y_train = np.array(y_train)
y_train_cat = to_categorical(y_train, num_classes=4)


In [ ]:
start_train_actor = 0
end_train_actor=5000

start_test_actor = -2000
end_test_actor = -1000

start_train_critic = 0
end_train_critic = 5000

start_test_critic = -2000
end_test_critic = -1000

critic_accuracy = []
actor_accuracy = []

ACTOR-CRITIC Loop

In [ ]:
for i in range(10):
  # X_train for Actor (X_pred for Critic)
  x_temp_train = X_train[start_train_actor:end_train_actor, :-2]
  X_train_actor = x_temp_train

  #X_test for Actor
  X_temp_test = X_train[start_test_actor:end_test_actor, :-2]
  X_test_actor = X_temp_test
  y_test = y_train[start_test_actor:end_test_actor]


  # Train data for Critic
  #ENTER PATH TO DATA
  X_train_temp2 = np.load("/content/drive/MyDrive/AI-Project-3/X_train150.npy")
  y_train_temp2 = np.load("/content/drive/MyDrive/AI-Project-3/y_train150.npy")

  #Slicing for epochs
  X_train_critic = X_train_temp2[start_train_critic:end_train_critic]
  X_test_critic = X_train_temp2[start_test_critic:end_test_critic]
  y_train_critic = y_train_temp2[start_train_critic:end_train_critic]
  y_test_critic = y_train_temp2[start_test_critic:end_test_critic]


  # Train the Critic model
  inp_size = X_train_critic.shape[1]
  # Create a logistic regression model
  critic_model = RandomForestClassifier(warm_start=True)
  critic_model.fit(X_train_critic[:, :inp_size], y_train_critic)
  # Prediction from Critic
  # Predict probabilities for class 1
  probabilities = critic_model.predict_proba(X_test_critic[:,:inp_size])[:, 1]

  pred = []
  for i in range(len(probabilities)):
    if probabilities[i]>= 0.5:
      pred.append(1)
    else:
      pred.append(0)

  c_accuracy = accuracy_score(y_test_critic, np.array(pred))
  critic_accuracy.append(c_accuracy)
  print("Critic Accuracy: ",c_accuracy)


  #Training data prediction loop
  preds = {}
  for i in range(0,4):
    move = np.full((len(x_temp_train), 1), i)
    X_input_critic = np.hstack((x_temp_train, move))
    preds[i] = list(critic_model.predict_proba(X_input_critic[:,:inp_size])[:, 1])


  # Training truth for Actor
  y_train_actor = []
  for i in range(len(X_input_critic)):
    temp = [preds[0][i], preds[1][i], preds[2][i], preds[3][i]]

    ans = temp.index(max(temp))
    y_train_actor.append(ans)
  y_train_actor = np.array(y_train_actor)


  # ACTOR
  inp_size2 = X_train_actor.shape[1]

  actor_model = LogisticRegression(multi_class='ovr', solver='lbfgs', max_iter=1000, warm_start=True)
  # Train the model on the training data
  actor_model.fit(X_train_actor[:,:inp_size2], y_train_actor)
  train_predictions = actor_model.predict(X_train_actor[:,:inp_size2])
  train_accuracy = accuracy_score(y_train_actor, train_predictions)
  print(f"Actor Accuracy (Train): {train_accuracy:.4f}")

  # Convert one-hot encoded predictions back to categorical labels
  y_pred = actor_model.predict(X_test_actor[:,:inp_size2])

  # Evaluate the model
  a_accuracy = accuracy_score(y_test, to_categorical(y_pred, num_classes=4))
  actor_accuracy.append(a_accuracy)
  print(f"Actor Accuracy (Test): {a_accuracy:.4f}")

  start_train_actor = end_train_actor
  end_train_actor += 5000

  start_train_critic += 1000
  end_train_critic += 1000

  end_test_actor = start_test_actor
  start_test_actor -= 1000

  end_test_critic = start_test_critic
  start_test_critic -= 1000